In [10]:
import pandas as pd

data2 = ['양파', '당근', '계란', '마늘', '무', '양배추', '깻잎', '파프리카']
df = pd.DataFrame({'ingredient' : data2})
df.to_csv('./output/냉장고재료test용.csv')

data = pd.read_csv('./output/test.csv', low_memory=False)
data2 = pd.read_csv('./output/냉장고재료test용.csv', low_memory=False)
s = ' '.join(data2['ingredient'])
data.loc[400] = ['냉장고재료',s,s,s,s]

In [28]:
test = pd.read_csv("./output/recipe.csv")



<class 'str'>


In [30]:
test["seasoning"].replace("[", "")
print(test["seasoning"][1])

[]


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# data = pd.read_csv('../smart_refrigerator/output/test.csv', low_memory=False)
data2 = pd.read_csv('../smart_refrigerator/output/냉장고재료test용.csv', low_memory=False)

data['ingredient'] = data['ingredint_name'].fillna('')
data2['ingredient'] = data2['ingredient'].fillna('')

tfidf = TfidfVectorizer()
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['ingredient'])
tfidf_matrix2 = tfidf.fit_transform(data2['ingredient'])

print(tfidf_matrix.shape)
print(tfidf_matrix2.shape)
# overview 열에 대해서 tf-idf를 수행했습니다. 20,000개의 영화를 표현하기위해 총 47,487개의 단어가 사용되었음을 보여주고 있습니다.
# 이제 코사인 유사도를 사용하면 바로 문서의 유사도를 구할 수 있습니다.

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# 코사인 유사도를 구합니다.

indices = pd.Series(data.index, index=data['food'])
print(indices.head())
# 영화의 타이틀과 인덱스를 가진 테이블을 만듭니다. 이 중 5개만 출력해보도록 하겠습니다.
# 이 테이블의 용도는 영화의 타이틀을 입력하면 인덱스를 리턴하기 위함입니다.


# 이제 선택한 영화에 대해서 코사인 유사도를 이용하여, 가장 overview가 유사한 10개의 영화를 찾아내는 함수를 만듭니다.
def get_recommendations(food, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[food]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['food'].iloc[movie_indices]

get_recommendations('냉장고재료')

(401, 455)
(8, 7)
food
소고기 버섯전골 레시피 맛있게 만드는 법                       0
완전맛있는 차돌박이찜! 미소된장마요소스도 함께~                   1
가지등심카나페                                      2
소고기두부볶음 - 아이들 반찬/이유식 반찬으로도 최고죠~옛생각이 절로...    3
담백한 차돌박이부추말이찜 레시피                            4
dtype: int64


217                                       소고기 덮밥~*
33                                      소고기 파프리카말이
104                                   한우 품은 오꼬나미야끼
187    소고기 양배추국 만드는 법, 요즘 딱 먹기 좋은 양배추로 간단 국물요리 만들기
304                  6대 영양소가 모두 들어있는 우리육우 찹스테이크 덮밥
189                               파프리카가 곁들어진 찹스테이크
112                           (간장양념) 차돌박이 볶음우동 만들기
177                              간단하지만 폼나는 소고기채소말이
263                          불고기전골 양념 만드는 법 손님초대요리
362                  암예방레시피 소고기 스튜 헝가리 굴라쉬 건강한 음식!
Name: food, dtype: object